# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [2]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [3]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [7]:
from itertools import combinations

classifier_classes = {LinearRegression, KNeighborsClassifier, SVC,
                  DecisionTreeClassifier, GaussianNB, QuadraticDiscriminantAnalysis}
    
def build_classifiers():
    for combination in combinations(classifier_classes, 3):
        classifiers = []
        for cls in combination:
            c = cls()
            c.fit(data_set, labels)
            classifiers.append(c)
        yield tuple(classifiers)

In [26]:
def build_stacked_classifier():
    for base_classifiers in build_classifiers():
        output = []
        for classifier in base_classifiers:
            output.append(classifier.predict(data_set))
        output = np.array(output).reshape((130,3))
    
        for stacked_classifier_cls in classifier_classes:
            stacked_classifier = stacked_classifier_cls()
            stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
            test_set = []
            for classifier in base_classifiers:
                test_set.append(classifier.predict(test_data_set))
            test_set = np.array(test_set).reshape((len(test_set[0]),3))
            predicted = stacked_classifier.predict(test_set)
            try:
                accuracy = accuracy_score(test_labels, predicted)
                print(f'Base: {", ".join(str(x.__class__) for x in base_classifiers)}')
                print(f'Stacked: {stacked_classifier_cls}')
                print(accuracy, '\n')
            except Exception as e:
                print(e)

In [27]:
build_stacked_classifier()

Classification metrics can't handle a mix of binary and continuous targets
Base: <class 'sklearn.linear_model._base.LinearRegression'>, <class 'sklearn.tree._classes.DecisionTreeClassifier'>, <class 'sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis'>
Stacked: <class 'sklearn.tree._classes.DecisionTreeClassifier'>
0.75 

Base: <class 'sklearn.linear_model._base.LinearRegression'>, <class 'sklearn.tree._classes.DecisionTreeClassifier'>, <class 'sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis'>
Stacked: <class 'sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis'>
0.2 

Base: <class 'sklearn.linear_model._base.LinearRegression'>, <class 'sklearn.tree._classes.DecisionTreeClassifier'>, <class 'sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis'>
Stacked: <class 'sklearn.svm._classes.SVC'>
0.6 

Base: <class 'sklearn.linear_model._base.LinearRegression'>, <class 'sklearn.tree._classes.DecisionTreeClassifier'>, <class 'sklearn.discriminant_analysis.Qua

/home/vlad_f/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/vlad_f/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/vlad_f/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/vlad_f/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Base: <class 'sklearn.linear_model._base.LinearRegression'>, <class 'sklearn.neighbors._classification.KNeighborsClassifier'>, <class 'sklearn.naive_bayes.GaussianNB'>
Stacked: <class 'sklearn.naive_bayes.GaussianNB'>
1.0 

Classification metrics can't handle a mix of binary and continuous targets
Base: <class 'sklearn.tree._classes.DecisionTreeClassifier'>, <class 'sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis'>, <class 'sklearn.svm._classes.SVC'>
Stacked: <class 'sklearn.tree._classes.DecisionTreeClassifier'>
0.15 

Base: <class 'sklearn.tree._classes.DecisionTreeClassifier'>, <class 'sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis'>, <class 'sklearn.svm._classes.SVC'>
Stacked: <class 'sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis'>
0.1 

Base: <class 'sklearn.tree._classes.DecisionTreeClassifier'>, <class 'sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis'>, <class 'sklearn.svm._classes.SVC'>
Stacked: <class 'sklearn.svm._classes.SVC

## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [30]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    return (1+Z)/1.0

Fill the two functions below:

In [37]:
def set_new_weights(model):
    w_new = (1 + ~(model.predict(test_set) == test_labels))\
    / np.sum(( 1 + ~(model.predict(test_set) == test_labels)))
    return w_new 

Train the classifier with the code below:

In [38]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

print(weights)


validate_x, validate_label = generate_data(1, dimension, labels)

[0.0006566 0.0006566 0.0013132 0.0013132 0.0006566 0.0006566 0.0006566
 0.0013132 0.0013132 0.0013132 0.0006566 0.0013132 0.0013132 0.0013132
 0.0013132 0.0013132 0.0006566 0.0006566 0.0006566 0.0013132 0.0013132
 0.0013132 0.0013132 0.0006566 0.0013132 0.0006566 0.0013132 0.0006566
 0.0006566 0.0013132 0.0013132 0.0006566 0.0006566 0.0013132 0.0013132
 0.0006566 0.0013132 0.0006566 0.0006566 0.0006566 0.0013132 0.0006566
 0.0013132 0.0013132 0.0013132 0.0006566 0.0013132 0.0013132 0.0006566
 0.0013132 0.0013132 0.0013132 0.0006566 0.0006566 0.0013132 0.0006566
 0.0013132 0.0013132 0.0013132 0.0006566 0.0006566 0.0013132 0.0013132
 0.0013132 0.0006566 0.0013132 0.0013132 0.0006566 0.0006566 0.0013132
 0.0006566 0.0013132 0.0013132 0.0013132 0.0013132 0.0006566 0.0006566
 0.0013132 0.0013132 0.0006566 0.0006566 0.0013132 0.0006566 0.0013132
 0.0013132 0.0013132 0.0013132 0.0006566 0.0006566 0.0006566 0.0006566
 0.0013132 0.0013132 0.0013132 0.0013132 0.0006566 0.0006566 0.0006566
 0.001

Set the validation data set:

In [45]:
validate_x, validate_label = generate_data(1, dimension, labels)
print(validate_x, validate_label)

[[0.76475116 0.57860552]] [1]


Fill the prediction code:

In [42]:
def get_prediction(x):
    predictions = []
    for classifier in classifiers:
        predictions.append(classifier.predict(test_set))
    predictions = np.asarray(predictions).reshape(len(classifiers), -1)
    final = []
    for prediction in predictions:
        final.append(np.argmax(np.bincount(prediction)))
    return final

Test it:

In [43]:
prediction = get_prediction(validate_x)[0]

print(prediction)

1
